# The Essentials Function to Calculate the Sub-Space Operators

In [15]:
using QuantumOptics

# https://juliapackages.com/p/einsum
using Einsum

In [16]:
"""
Compute the single-particle operator from single-particle matrix.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `N::Integer`: The total site number.
- `sp_matrix::Matrix`: Hopping phases matrix from any model.
"""
function get_sp_op(sp_basis, N, sp_matrix)
    
    H = SparseOperator(sp_basis)

    for m in 1:N
        for n in 1:N
            H += sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op

In [17]:
"""
Compute the single-particle operator from single-particle matrix.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `N::Integer`: The total site number.
- `sp_matrix::Matrix`: Hopping phases matrix from any model.
"""
function get_sp_op_data(sp_basis, N, sp_matrix)
  
    H = SparseOperator(sp_basis)

    for m in 1:N
        for n in 1:N
            H += sp_matrix.data[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op_data

In [3]:
"""
Compute the first eigen-states in a given number.

#### Arguments
- `sp_op::get_sp_op`: Single-particle operator.
- `cut_off::Int`: Limit value of eigen-state index.
"""
function get_sub_states(sp_op, cut_off)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[1:cut_off]
    
    return states
end

get_sub_states

In [18]:
"""
Compute sub-space basis, projection and comlex conjugate of projection operator.

#### Arguments
- `states::get_sub_states`: Eigen-states of the sub-space.
- `basis::NLevelBasis`: Single-particle basis.
"""
function get_projector_op(states, sp_basis)
    
    b_sub = SubspaceBasis(sp_basis,states)
    P = projector(b_sub, sp_basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_projector_op

In [19]:
"""
Compute the corresponding operator in the sub-space.

#### Arguments
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
- `P::get_projector_op[2]`: Projection operator.
- `Pt::get_projector_op[3]`: Complex conjugate of projection operator.
"""
function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end

get_subspace_op

In [20]:
"""
Compute the single-particle number operator for each lattice sites.

#### Arguments
- `N::Integer`: The total site number.
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `P::get_projector_op[2]`: Projection operator.
- `Pt::get_projector_op[3]`: Complex conjugate of projection operator.
"""
function get_num_sub_list(N, sp_basis, P, Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(sp_basis, m, m)
        NMP = get_subspace_op(NM, P, Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list

In [21]:
"""
Compute the many-body operator for boson particles from single-particle operator.

#### Arguments
- `basis_mb`: Many-body basis.
- `basis_sub::get_projector_op[1]`: Sub-space basis.
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
"""
function get_mb_op(basis_mb, basis_sub, sp_op)
    
    Op_mb = SparseOperator(basis_mb)
    
    for i in 1:length(basis_sub)
        for j in 1:length(basis_sub)
            Op_mb += sp_op.data[i,j] * transition(basis_mb, i, j)
        end
    end
    
    return Op_mb
end

get_boson_mb_op

In [22]:
# """
# Compute the Many-Body Number Operator for each lattice sites.

# #### Arguments
# - `basis_mb::ManyBodyBasis`: Created Many-Body Basis by using `bosonstates` or `fermionstates`.
# - `basis::SubSpaceBasis`: Created Sub-Space Basis by using get_projector_op.
# - `num_sub_list::ElementVector`: Created Single-Particle Number Number Operator for each lattice sites by using get_num_sub_list.
# """
# function get_num_mb_list(basis_mb, basis, num_sub_list)
    
#     num_mb_list = []
    
#     for m in 1:N
#         NMP = get_mb_op(basis_mb, basis, num_sub_list[m])
#         push!(num_mb_list, NMP)
#     end
    
#     return num_mb_list
# end

In [9]:
# function get_hubbard_int(num_op_list, basis_mb, U)

#     IT = SparseOperator(basis_mb)
    
#     for m in 1:N
#         IT += U/2 * ( num_op_list[m] * num_op_list[m] - num_op_list[m] )
#     end
    
#     return IT
# end

get_hubbard_int (generic function with 1 method)

In [10]:
# function get_hubbard_int2(basis, P, Pt)

#     basis2 = basis ⊗ basis

#     # interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i

#     Vint = SparseOperator(basis2)

#     for n in 1:N
#         Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
#     end

#     Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

#     Vint_mb = manybodyoperator(basis_mb, Vint_sub)
    
#     return Vint_mb
# end

get_hubbard_int2 (generic function with 1 method)

In [11]:
# function get_hubbard_int3(P, Pt, b_sub, cut_off)
   
#     P1 = P.data
#     P1t = Pt.data

#     @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

#     b2_sub = b_sub ⊗ b_sub

#     P4re = reshape(P4, cut_off^2, cut_off^2)

#     Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    
#     Vint_bsub2_mb = manybodyoperator(basis_mb, Vint_bsub2)
    
#     return Vint_bsub2_mb
# end

get_hubbard_int3 (generic function with 1 method)

In [12]:
# function get_hubbard_int4(P, Pt, b_sub, cut_off)
    
#     bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off, PN)

#     P1 = P.data
#     P1t = Pt.data;

#     @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

#     b2cut = bcut ⊗ bcut

#     P4re = reshape(P4, cut_off^2, cut_off^2)

#     Vint_bsub2 = SparseOperator(b2cut, U/2*P4re)
        
#     Vint_mb_cut = manybodyoperator(bcut_mb, Vint_bsub2)
    
#     return Vint_mb_cut
# end

get_hubbard_int4 (generic function with 1 method)

In [13]:
"""
Compute the on-site Bose-Hubbard Interaction term.

#### Arguments:
- `basis_cut_sp`: The cutt-off sites of bosonic many-body basis.
- `basis_cut_mb`: The cutt-off sites of single-particle basis.
- `P::Operator`: Projection operator.
- `Pt::Operator`: Complex conjugate of projection operator.
- `cut_off::Int`: Limit value of eigen-state index.
"""
function Hubbard_Interaction(basis_cut_sp, basis_cut_mb, P, Pt, cut_off)
    
    P1 = P.data
    P1t = Pt.data;

    @einsum Coefficient[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    Vint_mb_cut = SparseOperator(basis_cut_mb)
        
    for k in 1:cut_off
        for l in 1:cut_off
            for m in 1:cut_off
                for n in 1:cut_off
                    a1t = create(basis_cut_mb, k)
                    a2t = create(basis_cut_mb, l)
                    a2  = destroy(basis_cut_mb, m)      
                    a1  = destroy(basis_cut_mb, n)      
                    Vint_mb_cut += U/2*Coefficient[k,l,m,n]*a1t*a2t*a2*a1
                end
            end
        end
    end
    
    return Vint_mb_cut
end

Hubbard_Interaction

In [ ]:
"""
Compute the many-body number operators for each sites.

#### Arguments:
- `basis_cut_sp`: The cutt-off sites of bosonic many-body basis.
- `num_sub_list::get_num_sub_list`: Single-particle number operators.
- `basis_cut_mb`: The cutt-off sites of single-particle basis.
"""
function get_num_mb_op(N, basis_cut_sp, num_sub_list, basis_cut_mb, basis_sub)
    
    number_sp_list_operators = []
    for i in 1:N
        number_sp_list = Operator(basis_cut_sp, num_sub_list[i].data)
        push!(number_sp_list_operators, number_sp_list)
    end
    
    number_mb_list_operators = []
    for i in 1:N
        number_mb_list = get_mb_op(basis_cut_mb, basis_sub, number_sp_list_operators[i])
        # number_mb_list = manybodyoperator(bcut_mb, number_sp_list_operators[i])
        push!(number_mb_list_operators, number_mb_list)
    end
    
    return number_mb_list_operators
end